Warmer temperatures also seem to be the culprit of black bears' problems. Bears prefer cooler temperatures, particularly in winter for hybernation. They prefer to live in a place with rain so that the soil is moist and supports dense vegetation, and there are water sources. 

In [13]:
#| code-fold: true
#| warning: false

import pandas as pd 

data = pd.read_csv("../../../data/NABR_historic.csv")

data_colmap = {
    "long" : "longitude",
    "lat" : "latitude",
    "TimePeriod" : "time_period",
    "RCP" : "rep_con_path",
    "treecanopy" : "pct_treecanopy", 
    "Ann_Herb" : "pct_annual_herbcov",
    "Bare" : "pct_bareground",
    "Herb" : "pct_herbcov",
    "Litter" : "pct_plantlittercov",
    "Shrub" : "pct_shrubcov",
    "DrySoilDays_Summer_whole" : "drysoildays_summer",
    "Evap_Summer" : "evap_summer", 
    "ExtremeShortTermDryStress_Summer_whole" : "xtr_st_drysoilstress_summer", 
    "FrostDays_Winter" : "frostdays_winter", 
    "NonDrySWA_Summer_whole" : "nondry_swa_summer",
    "PPT_Winter" : "precipitation_winter", 
    "PPT_Summer" : "precipitation_summer",
    "PPT_Annual" : "precipitation_annual", 
    "T_Winter" : "temp_winter",
    "T_Summer" : "temp_summer", 
    "T_Annual" : "temp_annual", 
    "Tmax_Summer" : "maxtemp_summer", 
    "Tmin_Winter" : "mintemp_winter", 
    "VWC_Winter_whole" : "volwater_winter", 
    "VWC_Spring_whole" : "volwater_spring", 
    "VWC_Summer_whole" : "volwater_summer", 
    "VWC_Fall_whole" : "volwater_fall"
}

data = data.rename(columns=data_colmap)

soil_data_cols = ['year', 'pct_treecanopy', 'pct_annual_herbcov', 'pct_bareground',
       'pct_herbcov', 'pct_plantlittercov', 'pct_shrubcov',
       'drysoildays_summer', 'evap_summer', 'xtr_st_drysoilstress_summer'] 

soil_data = data[soil_data_cols]

soil_data_agg = soil_data.groupby("year")[['pct_treecanopy', 'pct_annual_herbcov', 'pct_bareground',
       'pct_herbcov', 'pct_plantlittercov', 'pct_shrubcov',
       'drysoildays_summer', 'evap_summer', 'xtr_st_drysoilstress_summer']].agg("mean").reset_index()

data_agg = data.groupby("year")[['pct_treecanopy', 'pct_annual_herbcov', 'pct_bareground',
       'pct_herbcov', 'pct_plantlittercov', 'pct_shrubcov',
       'drysoildays_summer', 'evap_summer', 'xtr_st_drysoilstress_summer',
       'frostdays_winter', 'nondry_swa_summer', 'precipitation_winter',
       'precipitation_summer', 'precipitation_annual', 'temp_winter',
       'temp_summer', 'temp_annual', 'maxtemp_summer', 'mintemp_winter',
       'volwater_winter', 'volwater_spring', 'volwater_summer',
       'volwater_fall']].agg('mean').reset_index()

In [2]:
import plotly.express as px

px.scatter(soil_data_agg, x = "year", y = ['pct_treecanopy', 'pct_annual_herbcov', 'pct_bareground',
       'pct_herbcov', 'pct_plantlittercov', 'pct_shrubcov',
       'drysoildays_summer', 'evap_summer', 'xtr_st_drysoilstress_summer'], trendline = "ols")

/Users/reneedemaio/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning:

urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020



In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

columns = ['pct_treecanopy', 'pct_annual_herbcov', 'pct_bareground',
       'pct_herbcov', 'pct_plantlittercov', 'pct_shrubcov',
       'drysoildays_summer', 'evap_summer', 'xtr_st_drysoilstress_summer']

app.layout = html.Div([
    html.H1("Interactive Soil and Coverage Data Graph"),
    
    html.Div([
        html.Label("Select Y-axis:"),
        dcc.Dropdown(
            id='y-axis-dropdown',
            options=[{'label': col, 'value': col} for col in columns],
            value='pct_treecanopy', 
            multi = True
        ),
    ]),
    
    dcc.Graph(id='scatter-plot')
])

@app.callback(
    Output('scatter-plot', 'figure'),
     Input('y-axis-dropdown', 'value')
)
def update_graph(y_axis):
    fig = px.scatter(data_agg, x="maxtemp_summer", y=y_axis, title=f'{y_axis} vs Year', trendline = "ols")
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port = 8052)


In [4]:
temp_data = pd.read_csv("../../../data/cleaned_temp_subset.csv")

temp_data = temp_data.drop(columns="Unnamed: 0")

temp_soil_data = pd.merge(temp_data, soil_data_agg, on = "year", how = "outer")

In [5]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

columns = ['pct_treecanopy', 'pct_annual_herbcov', 'pct_bareground',
       'pct_herbcov', 'pct_plantlittercov', 'pct_shrubcov',
       'drysoildays_summer', 'evap_summer', 'xtr_st_drysoilstress_summer',
       'frostdays_winter', 'nondry_swa_summer', 'precipitation_winter',
       'precipitation_summer', 'precipitation_annual', 'temp_winter',
       'temp_summer', 'temp_annual', 'maxtemp_summer', 'mintemp_winter',
       'volwater_winter', 'volwater_spring', 'volwater_summer',
       'volwater_fall']

app.layout = html.Div([
    html.H1("Interactive Temperature and Soil Data Graph"),
    
    html.Div([
        html.Label("Select Y-axis:"),
        dcc.Dropdown(
            id='y-axis-dropdown',
            options=[{'label': col, 'value': col} for col in columns],
            value='pct_treecanopy', 
            multi = True
        ),
    ]),
    
    dcc.Graph(id='scatter-plot')
])

@app.callback(
    Output('scatter-plot', 'figure'),
     Input('y-axis-dropdown', 'value')
)
def update_graph(y_axis):
    fig = px.line(temp_soil_data, x="year", y=y_axis, title=f'{y_axis} vs Year')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port = 8053)
